In [1]:
from nupack import *

In [1]:
my_model = Model(material='dna', celsius=23)
def generate_conserved_ss_seqs(ss, seq_num):
    # Define sequence domains
    seq_len = len(ss)
    da = Domain(f'N{seq_len}', name='da')
    # Define strands containing these domains

    sa = TargetStrand([da], name='sa')

    # Define a target complex
    cstickfigure = TargetComplex([sa], ss, name='cstickfigure')
    #Define a target test tube containing all off-target complexes of up to 4 strands
    tstickfigure = TargetTube(on_targets={cstickfigure: 1e-06}, off_targets=SetSpec(max_size=1), name='tfigure')
    # Set a stop condition of 1% and a seed for random number generation to get a reproducible result for this demo
    my_options = DesignOptions(f_stop=0.01, seed=93)
    # Define and run the test tube design job
    my_design = tube_design(tubes=[tstickfigure], model=my_model, options=my_options)
    my_results = my_design.run(trials=seq_num)
    return list(set([str(list(x.domains.items())[0][1]) for x in my_results]))

NameError: name 'Model' is not defined

In [15]:
short_hair = "((((((......))))))"  # 18 ln
short_hair2 = "((((((....))))))" # 16 ln
gap_hair = "(((((((....))...)))))" # 21 ln

sh_seqs = generate_conserved_ss_seqs(short_hair, 2000)
sh2_seqs = generate_conserved_ss_seqs(short_hair2, 2000)
gap_seqs = generate_conserved_ss_seqs(gap_hair, 2000)

In [17]:
print(len(sh_seqs), len(sh2_seqs), len(gap_seqs))

1033 842 995


In [27]:
import numpy as np
seqs, labels = [], []
length = 26

seq_array = [sh_seqs, sh2_seqs, gap_seqs]
for did, dc in enumerate(seq_array):
    motif_len = len(dc[0])
    mps = np.random.randint(0, length-motif_len+1, len(dc))
    sls = mps + motif_len
    for i in range(len(dc)):
        # random sequence
        seq = "".join([np.random.choice(["A", "C", "G", "T"]) for _ in range(sls[i])])
        # put motif in position
        seq = seq[:mps[i]] + dc[i] + seq[mps[i]+motif_len:]
        # seq[mps[i]: mps[i] + len(motif)] = motif
        # adjust len of seq
        if len(seq) != length:
            seq += "".join(["-" for _ in range(length - len(seq))])
        seqs.append(seq)
        labels.append(did)

noise_num = 300
rand_lens = np.random.randint(16, 27, noise_num)
noise_label = len(seq_array)
for i in range(noise_num):
    seq = "".join([np.random.choice(["A", "C", "G", "T"]) for _ in range(rand_lens[i])])
    if len(seq) != length:
            seq += "".join(["-" for _ in range(length - len(seq))])
    seqs.append(seq)
    labels.append(noise_label)

In [28]:
from rbm_torch.utils.data_prep import write_fasta

write_fasta(seqs, labels, "toy_ss.fasta")

In [22]:
i

0